In [50]:
pip install facenet-pytorch mtcnn opencv-python numpy scikit-learn

In [73]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
from facenet_pytorch import InceptionResnetV1, MTCNN
import torch

# Initialize MTCNN (face detector) and InceptionResnetV1 (embedding model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
detector = MTCNN(image_size=160, margin=20, device=device)
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Dataset
dataset_path = "Dataset"
embeddings_dict = {}

# Extracting the embeddings
def extract_embedding(img_path):
    try:
        img = Image.open(img_path).convert('RGB')
        face = detector(img)
        if face is not None:
            face = face.unsqueeze(0).to(device)
            emb = model(face)
            return emb.detach().cpu().numpy()[0]
        else:
            print(f"❌ No face detected in {img_path}")
            return None
    except Exception as e:
        print(f"❌ Error in {img_path}: {e}")
        return None

# Loop through each student's folder
for student_id in tqdm(os.listdir(dataset_path)):
    student_folder = os.path.join(dataset_path, student_id)

    # Skip is required
    if not os.path.isdir(student_folder) or student_id.startswith("."):
        continue

    embeddings = []
    for img_name in os.listdir(student_folder):
        if img_name.startswith('.'):  # Skip hidden/system files
            continue
        img_path = os.path.join(student_folder, img_name)
        emb = extract_embedding(img_path)
        if emb is not None:
            embeddings.append(emb)

    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
        embeddings_dict[student_id] = avg_embedding
    else:
        print(f"⚠️ No valid images found for {student_id}")

# Save the embeddings
np.save("student_embeddings_02.npy", embeddings_dict)
print("✅ Embeddings saved successfully as student_embeddings.npy")


  0%|          | 0/6 [00:00<?, ?it/s]

❌ No face detected in Dataset\2241013015\72.jpg
❌ No face detected in Dataset\2241013015\76.jpg
❌ No face detected in Dataset\2241013015\77.jpg
❌ No face detected in Dataset\2241013015\78.jpg
❌ No face detected in Dataset\2241013015\79.jpg


 67%|██████▋   | 4/6 [01:12<00:39, 19.65s/it]

❌ No face detected in Dataset\2241016078\77.jpg


100%|██████████| 6/6 [02:21<00:00, 23.60s/it]

✅ Embeddings saved successfully as student_embeddings.npy


In [44]:
import torch
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from facenet_pytorch import MTCNN, InceptionResnetV1
import os

# Load model and embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(image_size=160, margin=0, keep_all=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Student embeddings loading
student_data = np.load("student_embeddings.npy", allow_pickle=True)
student_embeddings = student_data.item()

# Group image loading
group_img_path = "WhatsApp Image 2025-08-02 at 00.06.04_b696b984.jpg"  # replace with your image path
img = Image.open(group_img_path).convert('RGB')

# Detect faces
boxes, probs = mtcnn.detect(img)

if boxes is None:
    
    print("❌ No faces detected.")
else:
    print(f"✅ Detected {len(boxes)} face(s).")

present_students = []

# Drawing object
draw = ImageDraw.Draw(img)
font = ImageFont.load_default()

# Processing each detected face
for i, box in enumerate(boxes):
    x1, y1, x2, y2 = map(int, box)
    face_crop = img.crop((x1, y1, x2, y2)).resize((160, 160))
    face_tensor = torch.tensor(np.array(face_crop)).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0

    with torch.no_grad():
        embedding = resnet(face_tensor)

    # Comparing with all stored embeddings
    matched = False
    for reg_num, emb in student_embeddings.items():
        emb_tensor = torch.tensor(emb).unsqueeze(0).to(device).float()
        dist = torch.norm(embedding - emb_tensor).item()
        print(f"🧪 Comparing with {reg_num}: distance = {dist:.4f}")

        if dist < 0.9:  # Adjust threshold if needed
            print(f"✅ Match found: {reg_num}")
            draw.rectangle([x1, y1, x2, y2], outline="green", width=2)
            draw.text((x1, y2 + 2), reg_num, fill="green", font=font)
            present_students.append(reg_num)
            matched = True
            break

    if not matched:
        print(f"⚠️ No match for face #{i + 1}")
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        draw.text((x1, y2 + 2), "Unknown", fill="red", font=font)


img.save("output_with_boxes.jpg")
img.show()

print("\n✅ Present Students:")
print(list(set(present_students)))


✅ Detected 6 face(s).
🧪 Comparing with 2241013015: distance = 1.2025
🧪 Comparing with 2241016042: distance = 1.0022
⚠️ No match for face #1
🧪 Comparing with 2241013015: distance = 1.0809
🧪 Comparing with 2241016042: distance = 1.2182
⚠️ No match for face #2
🧪 Comparing with 2241013015: distance = 0.9780
🧪 Comparing with 2241016042: distance = 1.0970
⚠️ No match for face #3
🧪 Comparing with 2241013015: distance = 1.0948
🧪 Comparing with 2241016042: distance = 0.6728
✅ Match found: 2241016042
🧪 Comparing with 2241013015: distance = 0.7581
✅ Match found: 2241013015
🧪 Comparing with 2241013015: distance = 1.2917
🧪 Comparing with 2241016042: distance = 1.2635
⚠️ No match for face #6

✅ Present Students:
['2241016042', '2241013015']


In [42]:
import torch
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from facenet_pytorch import MTCNN, InceptionResnetV1
import os

# Load model and embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(image_size=160, margin=0, keep_all=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Student embeddings loading
student_data = np.load("student_embeddings.npy", allow_pickle=True)
student_embeddings = student_data.item()

# Group image loading
group_img_path = "WhatsApp Image 2025-08-02 at 00.06.05_dc1245d5.jpg"  # replace with your image path
img = Image.open(group_img_path).convert('RGB')

# Detect faces
boxes, probs = mtcnn.detect(img)

if boxes is None:
    print("❌ No faces detected.")
else:
    print(f"✅ Detected {len(boxes)} face(s).")

present_students = []

# Drawing object
draw = ImageDraw.Draw(img)
font = ImageFont.load_default()

# Processing each detected face
for i, box in enumerate(boxes):
    x1, y1, x2, y2 = map(int, box)
    face_crop = img.crop((x1, y1, x2, y2)).resize((160, 160))
    face_tensor = torch.tensor(np.array(face_crop)).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0

    with torch.no_grad():
        embedding = resnet(face_tensor)

    # Comparing with all stored embeddings
    matched = False
    for reg_num, emb in student_embeddings.items():
        emb_tensor = torch.tensor(emb).unsqueeze(0).to(device).float()
        dist = torch.norm(embedding - emb_tensor).item()
        print(f"🧪 Comparing with {reg_num}: distance = {dist:.4f}")

        if dist < 0.9:  # Adjust threshold if needed
            print(f"✅ Match found: {reg_num}")
            draw.rectangle([x1, y1, x2, y2], outline="green", width=2)
            draw.text((x1, y2 + 2), reg_num, fill="green", font=font)
            present_students.append(reg_num)
            matched = True
            break

    if not matched:
        print(f"⚠️ No match for face #{i + 1}")
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        draw.text((x1, y2 + 2), "Unknown", fill="red", font=font)


img.save("output_with_boxes.jpg")
img.show()

print("\n✅ Present Students:")
print(list(set(present_students)))


✅ Detected 6 face(s).
🧪 Comparing with 2241013015: distance = 1.1444
🧪 Comparing with 2241016042: distance = 1.1129
⚠️ No match for face #1
🧪 Comparing with 2241013015: distance = 1.1769
🧪 Comparing with 2241016042: distance = 1.2346
⚠️ No match for face #2
🧪 Comparing with 2241013015: distance = 0.6309
✅ Match found: 2241013015
🧪 Comparing with 2241013015: distance = 1.1101
🧪 Comparing with 2241016042: distance = 1.3723
⚠️ No match for face #4
🧪 Comparing with 2241013015: distance = 1.0702
🧪 Comparing with 2241016042: distance = 0.5295
✅ Match found: 2241016042
🧪 Comparing with 2241013015: distance = 0.8799
✅ Match found: 2241013015

✅ Present Students:
['2241016042', '2241013015']


In [46]:
import torch
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from facenet_pytorch import MTCNN, InceptionResnetV1
import os

# Load model and embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(image_size=160, margin=0, keep_all=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Student embeddings loading
student_data = np.load("student_embeddings.npy", allow_pickle=True)
student_embeddings = student_data.item()

# Group image loading
group_img_path = "WhatsApp Image 2025-08-02 at 00.13.27_6693aeba.jpg"  # replace with your image path
img = Image.open(group_img_path).convert('RGB')

# Detect faces
boxes, probs = mtcnn.detect(img)

if boxes is None:
    print("❌ No faces detected.")
else:
    print(f"✅ Detected {len(boxes)} face(s).")

present_students = []

# Drawing object
draw = ImageDraw.Draw(img)
font = ImageFont.load_default()

# Processing each detected face
for i, box in enumerate(boxes):
    x1, y1, x2, y2 = map(int, box)
    face_crop = img.crop((x1, y1, x2, y2)).resize((160, 160))
    face_tensor = torch.tensor(np.array(face_crop)).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0

    with torch.no_grad():
        embedding = resnet(face_tensor)

    # Comparing with all stored embeddings
    matched = False
    for reg_num, emb in student_embeddings.items():
        emb_tensor = torch.tensor(emb).unsqueeze(0).to(device).float()
        dist = torch.norm(embedding - emb_tensor).item()
        print(f"🧪 Comparing with {reg_num}: distance = {dist:.4f}")

        if dist < 0.9:  # Adjust threshold if needed
            print(f"✅ Match found: {reg_num}")
            draw.rectangle([x1, y1, x2, y2], outline="green", width=2)
            draw.text((x1, y2 + 2), reg_num, fill="green", font=font)
            present_students.append(reg_num)
            matched = True
            break

    if not matched:
        print(f"⚠️ No match for face #{i + 1}")
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        draw.text((x1, y2 + 2), "Unknown", fill="red", font=font)


img.save("output_with_boxes.jpg")
img.show()

print("\n✅ Present Students:")
print(list(set(present_students)))


✅ Detected 2 face(s).
🧪 Comparing with 2241013015: distance = 0.9143
🧪 Comparing with 2241016042: distance = 1.0670
⚠️ No match for face #1
🧪 Comparing with 2241013015: distance = 1.1122
🧪 Comparing with 2241016042: distance = 0.6976
✅ Match found: 2241016042

✅ Present Students:
['2241016042']


In [75]:
import torch
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from facenet_pytorch import MTCNN, InceptionResnetV1
import os

# Load model and embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(image_size=160, margin=0, keep_all=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Student embeddings loading
student_data = np.load("student_embeddings_02.npy", allow_pickle=True)
student_embeddings = student_data.item()

# Group image loading
group_img_path = "picnic01.jpg"  # replace with your image path
img = Image.open(group_img_path).convert('RGB')

# Detect faces
boxes, probs = mtcnn.detect(img)

if boxes is None:
    print("❌ No faces detected.")
else:
    print(f"✅ Detected {len(boxes)} face(s).")

present_students = []

# Drawing object
draw = ImageDraw.Draw(img)
font = ImageFont.load_default()

# Processing each detected face
for i, box in enumerate(boxes):
    x1, y1, x2, y2 = map(int, box)
    face_crop = img.crop((x1, y1, x2, y2)).resize((160, 160))
    face_tensor = torch.tensor(np.array(face_crop)).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0

    with torch.no_grad():
        embedding = resnet(face_tensor)

    # Comparing with all stored embeddings
    matched = False
    for reg_num, emb in student_embeddings.items():
        emb_tensor = torch.tensor(emb).unsqueeze(0).to(device).float()
        dist = torch.norm(embedding - emb_tensor).item()
        print(f"🧪 Comparing with {reg_num}: distance = {dist:.4f}")

        if dist < 0.9:  # Adjust threshold if needed
            print(f"✅ Match found: {reg_num}")
            draw.rectangle([x1, y1, x2, y2], outline="green", width=2)
            draw.text((x1, y2 + 2), reg_num, fill="green", font=font)
            present_students.append(reg_num)
            matched = True
            break

    if not matched:
        print(f"⚠️ No match for face #{i + 1}")
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        draw.text((x1, y2 + 2), "Unknown", fill="red", font=font)


img.save("output_with_boxes.jpg")
img.show()

print("\n✅ Present Students:")
print(list(set(present_students)))


✅ Detected 21 face(s).
🧪 Comparing with 2241013015: distance = 1.0787
🧪 Comparing with 22410160124: distance = 1.1343
🧪 Comparing with 2241016042: distance = 0.5991
✅ Match found: 2241016042
🧪 Comparing with 2241013015: distance = 1.2990
🧪 Comparing with 22410160124: distance = 1.1951
🧪 Comparing with 2241016042: distance = 0.9828
🧪 Comparing with 2241016056: distance = 1.0470
🧪 Comparing with 2241016078: distance = 1.1022
🧪 Comparing with 2241016088: distance = 0.8192
✅ Match found: 2241016088
🧪 Comparing with 2241013015: distance = 1.1686
🧪 Comparing with 22410160124: distance = 1.0066
🧪 Comparing with 2241016042: distance = 1.0131
🧪 Comparing with 2241016056: distance = 1.1401
🧪 Comparing with 2241016078: distance = 0.8659
✅ Match found: 2241016078
🧪 Comparing with 2241013015: distance = 1.2133
🧪 Comparing with 22410160124: distance = 0.9282
🧪 Comparing with 2241016042: distance = 0.9247
🧪 Comparing with 2241016056: distance = 0.8729
✅ Match found: 2241016056
🧪 Comparing with 224101